# TAREA 9 - DASK

## Alejandro Hernández Farías, Clave Única 87806

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo. Hacer lo mismo, pero utilizando la biblioteca Dask-ML

In [229]:
# Importamos las librerías que vamos a utilizar:
from dask import delayed
from dask import dataframe
import numpy as np
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [139]:
# Leemos la base de datos y creamos la columna de proporción de propinas en un viaje, pues el la que queremos predecir:
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df['prop_viaje']=trips_df['tip_amount']/trips_df['fare_amount']
trips_df=trips_df.fillna(trips_df.mean()).compute()
trips_df.head(3)

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,prop_viaje
0,A,22.0,1,1,4.6,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091
1,A,9.0,1,1,0.0,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000
2,A,7.5,1,1,1.0,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333


In [192]:
# Definimos las variable dependiente "y" y las independientes "X"
# Vamos a predecir la proporcion de propinas en funcion de la tarifa, el número de pasajeros, el id del taxi y la distancia del viaje.
X=np.array(trips_df[['fare_amount','passenger_count','taxi_id','trip_distance']])
y=np.array(trips_df[['prop_viaje']])

# Dividimos en train(70%) y test(30%). 
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3,random_state=1)

#### SECUENCIAL

In [216]:
%%time
# Definimos los parametros para hacer el grid search usando GridSearchCV.
modelo=LinearRegression()
param={'fit_intercept':[True,False],'normalize':[True,False],'copy_X':[True,False]}
grid_s=GridSearchCV(modelo,param,cv=5)

# Entrenamos el modelo utilizando los datos de entrenamiento.
grid_s.fit(X_train,y_train)

# Hacemos la predicción utilizando los datos de prueba para calcular la suma de diferencias al cuadarado.
suma_dif_cuadrado_s=np.mean((grid_s.predict(X_test)-y_test)**2)

# Imprimimos la suma de diferencias al cuadrado
print(suma_dif_cuadrado_s)

0.015384485022342511
CPU times: user 170 ms, sys: 100 ms, total: 270 ms
Wall time: 160 ms


#### PARALELO

In [218]:
%%time
# Definimos los parametros para hacer el grid search usando GridSearchCV.
modelo=LinearRegression()
param={'fit_intercept':[True,False],'normalize':[True,False],'copy_X':[True,False]}
grid_p=delayed(GridSearchCV(modelo,param,cv=5))

# Entrenamos el modelo utilizando los datos de entrenamiento.
grid_p.fit(X_train,y_train)

# Hacemos la predicción utilizando los datos de prueba para calcular la suma de diferencias al cuadarado.
suma_dif_cuadrado_p=np.mean((grid_p.predict(X_test)-y_test)**2)

# Imprimimos la suma de diferencias al cuadrado
print(suma_dif_cuadrado_p)

Delayed('mean-85555ca1-c1e8-426d-8715-c9935ce11c41')
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 4.29 ms


__- ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?__

__Es 37 veces más rápido. Lo anterior, debido a que el paralelo tardó 4.29 ms, mientras que el secuencial tardó 160 ms__

#### UTILIZANDO DASK-ML

In [227]:
%%time
from dask_searchcv import GridSearchCV
# Definimos los parametros para hacer el grid search usando GridSearchCV.
modelo=LinearRegression()
param={'fit_intercept':[True,False],'normalize':[True,False],'copy_X':[True,False]}
grid_d=GridSearchCV(modelo,param,cv=5)

# Entrenamos el modelo utilizando los datos de entrenamiento.
grid_d.fit(X_train,y_train)

# Hacemos la predicción utilizando los datos de prueba para calcular la suma de diferencias al cuadarado.
suma_dif_cuadrado_d=np.mean((grid_d.predict(X_test)-y_test)**2)

# Imprimimos la suma de diferencias al cuadrado
print(suma_dif_cuadrado_d)


0.015384485022342511
CPU times: user 150 ms, sys: 150 ms, total: 300 ms
Wall time: 229 ms


__- ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?__

__La ejecución utilizando Dask ML es la más lenta de la las tres pues tarda 229 ms, mientras que la secuencial tarda 160 ms y la paralelo 4.29 ms__